In [1]:
import json

from pathlib import Path
import os
import re
import subprocess
import sys

from IPython.display import display
import pandas as pd
import rich
import rich.console
import rich.table
import tqdm

In [2]:
def dict_to_pandas(d):
    return pd.DataFrame({"name": list(d.keys()), "version": list(d.values())})

def parse(text):
    reqs_text = [x.strip().replace(" ", "") for x in text.split("\n")]
    reqs_text.sort()
    deps = {}

    for line in reqs_text:
        output = re.split(r"==|<|@", line, 1)
        
        if len(output) == 2:
            package, version = output
            deps[package] = version
        else:
            assert len(output) == 1, f"bad - got {len(output)}: {line}"
            package = output[0]
            deps[package] = None
    return deps

pckg_deps = parse(Path("/home/mila/g/gagnonju/RL4LMs/requirements.txt").read_text().strip())
our_deps = parse(subprocess.check_output(["pip", "freeze"]).decode().strip())

In [13]:
name = []
version_a = []
version_b = []

for key in pckg_deps:
    if key in our_deps:
        name.append(key)
        version_a.append(pckg_deps[key])
        version_b.append(our_deps[key])

output_dict = {"name": name, "package": version_a, "pip": version_b}

table = rich.table.Table(*output_dict, title="Package versions")
for a, b, c in zip(*output_dict.values()):
    b_is_none = b is None
    b = str(b)
    c = str(c)
    if b.strip() == c.strip() or b_is_none:
        table.add_row(f"[green]{a}", f"[green]{b}", f"[green]{c}")
    else:
        table.add_row(f"[red bold]{a}", f"[red bold]{b}", f"[red bold]{c}")

rich.console.Console(width=80).print(table)


                                Package versions                                
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name               ┃ package                    ┃ pip                        ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ BLEURT             │ git+https://github.com/go… │ git+https://github.com/go… │
│ absl-py            │ None                       │ 1.3.0                      │
│ bert-score         │ 0.3.11                     │ 0.3.11                     │
│ datasets           │ None                       │ 2.7.0                      │
│ gem-metrics        │ git+https://github.com/GE… │ git+https://github.com/GE… │
│ gym                │ 0.21.0                     │ 0.21.0                     │
│ importlib-metadata │ 5.0                        │ 4.13.0                     │
│ jsonlines          │ None                       │ 3.1.0                      │
│ nltk               │ 3.7                        │ 3.7                        │
│ pandas             │ None                       │ 1.4.2                      │
│ py-rouge           │ 1.1                        │ 1.1                        │
│ rich               │ None                       │ 12.6.0                     │
│ sacrebleu          │ 2.2.0                      │ 2.2.0                      │
│ spacy              │ None                       │ 3.4.3                      │
│ stable-baselines3  │ 1.5.1a5                    │ 1.5.1a5                    │
│ torch              │ None                       │ 1.12.1+cu113               │
│ torchvision        │ None                       │ 0.13.1+cu113               │
│ tqdm               │ None                       │ 4.64.1                     │
│ transformers       │ None                       │ 4.24.0                     │
│ wandb              │ None                       │ 0.13.5                     │
└────────────────────┴────────────────────────────┴────────────────────────────┘